In [39]:

from sklearn.model_selection import train_test_split
#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from os import listdir
from PIL import Image
import glob, os
from os import listdir
from os.path import isfile, join
import numpy as np
from pandas import DataFrame
import pandas as pd
from matplotlib import pyplot as plt

In [40]:
# las funciones anteriores. Poner a parte e importar

In [41]:
def resize_image(imagen,size,output=False):
    im = Image.open(imagen)
    rgb_image = RGBA_a_RGB(im)
    im_rgb = rgb_image.resize(size)
    
    if not output:
        return im_rgb 
    onlyfiles = [f for f in listdir(output) if isfile(join(output, f))]
    if image not in onlyfiles:
        im.save(output)
        return im_rgb
    else:
        print("file already exists, continuing to next one") 
        return im_rgb

In [42]:
def RGBA_a_RGB(image1):
    if image1.mode == "RGBA":
        background = Image.new("RGB", image1.size, (255, 255, 255))
        background.paste(image1, mask = image1.split()[3])
        background.save(f"{image1}", "JPEG", quality=100)
        rgb_image = Image.open(f"{image1}")
        return rgb_image
    return image1

In [43]:
def image_to_array(image):
    im_np= np.asarray(image)
    one_line=im_np.ravel()
    return one_line

In [44]:
paths={"black":"INPUT/black/",
      "white":"INPUT/white/",
      "green":"INPUT/green/",
      "red":"INPUT/red/"}
images={}
for color,path in paths.items():
    images[color]=[]
    for im in listdir(path):
        if im.endswith("jpg"):
            #size=(64,64)
            a=resize_image(path+im,(64,64))
            images[color].append(np.asarray(a))
    images[color]=np.array(images[color])

In [45]:
def image_to_array(image):
    im_np= np.asarray(image)
    one_line=im_np.ravel()
    return one_line

In [46]:
X= np.concatenate(list(images.values()),axis=0)
y = np.concatenate([[k]* v.shape[0] for k,v in images.items()])

In [ ]:
# Aqui empieza realmente. Division entre la x y la y para test

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [48]:
img_width, img_height = 64, 64
img_rows, img_cols = 64, 64

# Prepare data to feed the NN
num_classes = 4

# Ask keras which format to use depending on used backend and arrange data as expected
if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = x_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_width, img_height)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_width, img_height, 3)

# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = tf.keras.utils.normalize(X_train,axis=1)
X_test = tf.keras.utils.normalize(X_test,axis=1)
#X_train = X_train.astype('float32') / 255
#X_test = X_test.astype('float32') / 255

print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

x_train shape: (8388, 64, 64, 3)
8388 train samples
2097 test samples


In [67]:
img_width, img_height = 64, 64

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(tf.keras.layers.Flatten())

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
# Ytrain
y_train=pd.get_dummies(pd.Series(y_train)).to_numpy()

In [ ]:
y_test=pd.get_dummies(pd.Series(y_test)).to_numpy()


In [76]:
y_test[1]

array([0, 0, 0, 1], dtype=uint8)

In [68]:
batch_size = 100
epochs = 20

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))



Epoch 1/20
84/84 [==============================] - 20s 239ms/step - loss: 0.3595 - accuracy: 0.6903 - val_loss: 0.2846 - val_accuracy: 0.7320
Epoch 2/20
84/84 [==============================] - 20s 234ms/step - loss: 0.2223 - accuracy: 0.8339 - val_loss: 0.1589 - val_accuracy: 0.8794
Epoch 3/20
84/84 [==============================] - 20s 236ms/step - loss: 0.1718 - accuracy: 0.8767 - val_loss: 0.1106 - val_accuracy: 0.9156
Epoch 4/20
84/84 [==============================] - 19s 225ms/step - loss: 0.1207 - accuracy: 0.9171 - val_loss: 0.0785 - val_accuracy: 0.9456
Epoch 5/20
84/84 [==============================] - 18s 217ms/step - loss: 0.0907 - accuracy: 0.9392 - val_loss: 0.0902 - val_accuracy: 0.9294
Epoch 6/20
84/84 [==============================] - 18s 210ms/step - loss: 0.0794 - accuracy: 0.9461 - val_loss: 0.0522 - val_accuracy: 0.9619
Epoch 7/20
84/84 [==============================] - 18s 212ms/step - loss: 0.0664 - accuracy: 0.9572 - val_loss: 0.0445 - val_accuracy: 0.9666

KeyboardInterrupt: 

In [74]:
X_test[56].shape
model.predict(np.array([X_test[56]]))

array([[9.9997890e-01, 2.3024959e-06, 5.2825902e-07, 5.8759038e-09]],
      dtype=float32)

In [ ]:
cambiar el getdummies por el onehotencoder de felipe